## Подгрузка необходимых библиотек

In [ ]:
# Управление отображением 
from IPython import display  

# Работа с датами и временем
from datetime import datetime, timedelta  

# Работа с данными 
import pandas as pd  
import numpy as np  

# Визуализация
import matplotlib.pyplot as plt  
import seaborn as sns  

# Обработка текста (NLP)
import re  
import spacy  
import swifter  
from nltk.corpus import stopwords  
import nltk  
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  

# Машинное обучение
from sklearn.preprocessing import MaxAbsScaler  
from sklearn.linear_model import LogisticRegression  
from sklearn.svm import LinearSVC  
from xgboost import XGBClassifier  
from sklearn.metrics import classification_report, accuracy_score  
from sklearn.model_selection import train_test_split  

# Оптимизация гиперпараметров
import optuna  

# Управление предупреждениями
from warnings import filterwarnings
filterwarnings('ignore')  

## Обработка текста

В ходе обучения моделей для уменьшения размерности, а также для более корректных результатов, стоит провести обработку текста. Ниже была написана функция, которая приводит текст к нижнему регистру, удаляет лишние символы, проводит лемматизацию и удаляет стоп-слова. Однако из-за размерности данных, функция работает довольно долго. Поэтому для обучения и предсказаний моделей были заранее подготовлены два датасета с обработанными данными. Один тренировочный осонованный данных, которые удалось собрать, а другой контрольный, который используется для получения результатов на соревновании kaggle.

In [3]:
russian_stopwords = set(stopwords.words("russian"))
nlp = spacy.load("ru_core_news_sm")

def preprocess_text_spacy(text):

    # Удаление лишних символов и приведение текста к нижнему регистру
    text = re.sub(r"[^а-яА-ЯёЁ\s]", "", text.lower())
    
    # Лемматизация слов с помощью библиотеки spacy
    doc = nlp(text)
    
    # Лемматизация и удаление стоп-слов
    processed_words = [token.lemma_ for token in doc if token.lemma_ not in russian_stopwords and not token.is_punct]
    
    return " ".join(processed_words)


In [5]:
df = pd.read_csv('lenta_data_processed.csv', index_col=0)

## Обучение моделей

### Baseline

Для начала стоит задать baseline для предсказаний. Для этого обучим модель логистической регрессии на данных без какой-либо обработки с преобразованием в мешок слов.

In [164]:
X = df[['text']]
y = df.topic

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)

In [113]:
bow = CountVectorizer(min_df=0.00001) # подбор гиперпараметров очень помогает
bow.fit(X_train['text'])

bow_train = bow.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = bow.transform(X_test['text'])

print(bow_train.shape)

scaler = MaxAbsScaler()
bow_train = scaler.fit_transform(bow_train)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow_train, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))
print(accuracy_score(y_test, pred))

(55011, 281024)
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      5798
           1       0.92      0.88      0.90      2077
           2       0.92      0.90      0.91      2057
           3       0.89      0.90      0.89      3791
           4       1.00      0.99      0.99      1687
           5       0.95      0.95      0.95       655
           6       0.88      0.70      0.78       270
           7       0.95      0.90      0.92       878
           8       0.95      0.88      0.91      1125

    accuracy                           0.90     18338
   macro avg       0.92      0.89      0.91     18338
weighted avg       0.91      0.90      0.90     18338

0.9047878721779911


In [96]:
Test = pd.read_csv("test_news.csv")
bow_test_c = bow.transform(Test['content'])
bow_test_c = scaler.transform(bow_test_c)
preds = clf.predict(bow_test_c)
subm = pd.read_csv("base_submission_news.csv")
subm['topic'] = preds
subm.to_csv("baseline_lenta.csv", index=False)

Baseline модель показывает себя весьма неплохо с результатом `accuracy = 0.819` 

### Logistic Regression

Теперь будем обучать модели на обработанных данных с удалением стоп-слов и лемматизацией.

In [99]:
X = df[['processed_text']]
y = df.topic

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [101]:
bow_transformer = CountVectorizer(min_df=0.00001) # подбор гиперпараметров очень помогает
bow_transformer.fit(X_train['processed_text'])

bow_train = bow_transformer.transform(X_train['processed_text'])  # bow — bag of words (мешок слов)
bow_test = bow_transformer.transform(X_test['processed_text'])

print(bow_train.shape)

scaler = MaxAbsScaler()
bow_train = scaler.fit_transform(bow_train)
bow_test = scaler.transform(bow_test)

(55011, 148003)


In [70]:
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow_train, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))
print(accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      5798
           1       0.92      0.89      0.90      2077
           2       0.92      0.90      0.91      2057
           3       0.89      0.90      0.89      3791
           4       0.99      0.99      0.99      1687
           5       0.97      0.95      0.96       655
           6       0.83      0.76      0.79       270
           7       0.94      0.89      0.91       878
           8       0.95      0.86      0.90      1125

    accuracy                           0.90     18338
   macro avg       0.92      0.89      0.90     18338
weighted avg       0.90      0.90      0.90     18338

0.9033155196858982


Как бы странно не было, но модель показывает себя чуточку хуже на обработанных данных. При этом значение имеет парметр в `train_test_split(stratify=y)`. Если его не использовать, то модель дает чуть более метрику `accuracy` на обработанных данных.

Попробуем подобрать параметры модели так, чтобы она дала результат получше на обработанных данных.

#### Подбор параметров

In [ ]:
def log_reg_obj(trial):
    C = trial.suggest_loguniform('C', 1e-3, 1e2)  # Регуляризация
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])# Алгоритм оптимизации
    max_iter = trial.suggest_int('max_iter', 500, 5000)
    mc = trial.suggest_categorical('multi_class', ['ovr', 'multinomial'])
    cw = trial.suggest_categorical('class_weight', [None, 'balanced'])

    clf = LogisticRegression(
        max_iter=max_iter,
        random_state=42,
        multi_class=mc,
        class_weight=cw,
        C=C,
        solver=solver,
        n_jobs = -1
    )
    clf.fit(bow_train, y_train)
    
    pred = clf.predict(bow_test)
    accuracy = accuracy_score(y_test, pred)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(log_reg_obj, n_trials=25)

# Лучшие параметры и результат
print("Лучшие параметры:", study.best_params)
print("Лучшая точность:", study.best_value)

С относительно быстрым подбором гиперпараметров удалось все-таки получить чуть более лучшую метрику `accuracy`. Output был очищен, чтобы не перенагружать ноутбук.

In [81]:
params = {'C': 1.176273146130487, 'solver': 'lbfgs', 'max_iter': 4488, 'multi_class': 'ovr', 'class_weight': 'balanced'}

In [86]:
lr_clf = LogisticRegression(**params, random_state=42)
lr_clf.fit(bow_train, y_train)
pred = lr_clf.predict(bow_test)

print(classification_report(y_test, pred))
print(accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      5798
           1       0.91      0.91      0.91      2077
           2       0.90      0.92      0.91      2057
           3       0.88      0.90      0.89      3791
           4       0.99      0.99      0.99      1687
           5       0.95      0.96      0.96       655
           6       0.76      0.83      0.79       270
           7       0.92      0.93      0.92       878
           8       0.91      0.89      0.90      1125

    accuracy                           0.91     18338
   macro avg       0.90      0.91      0.91     18338
weighted avg       0.91      0.91      0.91     18338

0.9051150616206783


In [105]:
Test = pd.read_csv("test_news_processed.csv")
bow_test_c = bow_transformer.transform(Test['processed_content'])
bow_test_c = scaler.transform(bow_test_c)
preds = lr_clf.predict(bow_test_c)
subm['topic'] = preds
subm.to_csv("lr_lenta.csv", index=False)

После тестирования на контрольных  данных с соревнования был замечен весьма странный результат. Не смотря на то, что по метрикам в ноутбуке все стало получше в плане метрик, в соревновании результат получился хуже с метрикой `accuracy = 0.809`. В какой-то степени исходя из этого возможно стоит использовать данные без удаления слов и лемматизации. 

По предыдущим тестам, которые были сделаны в черновом ноутбуке, часть которых появится и в этом, преобразование TF-IDF не дало особо хороших результатов на контрольных данных с соревнования. Не смотря на то, что результаты по метрикам на собранных данных довольно значительно повышаются, на контрольных данных метрики понижаются. При этом всем лучше всего на соревновании показала себя модель XGBoost обученная на мешке слов с обработанными данными. Учитывая то, что результаты на соревновании kaggle понижаются на обработанных данных, стоит порпобовать обучить эту модель на необработанных данных.

### XGBoost

In [ ]:
X = df[['text']]
y = df.topic

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)

bow = CountVectorizer(min_df=0.00001) 
bow.fit(X_train['text'])

bow_train = bow.transform(X_train['text'])  
bow_test = bow.transform(X_test['text'])

print(bow_train.shape)

scaler = MaxAbsScaler()
bow_train = scaler.fit_transform(bow_train)
bow_test = scaler.transform(bow_test)

In [115]:
params = {'n_estimators': 946, 'max_depth': 25, 'learning_rate': 0.02334486067381037,
          'subsample': 0.7571222430291574, 'colsample_bytree': 0.6866556282657983,
          'gamma': 0.09038470099513807}

xg_clf = XGBClassifier(**params, random_state=42, scale_pos_weight = len(y_train) / (len(set(y_train)) * np.bincount(y_train)),
                        objective = "multi:softmax", num_class = len(set(y_train)), use_label_encoder = False)
xg_clf.fit(bow_train, y_train)
pred = xg_clf.predict(bow_test)

print(classification_report(y_test, pred))
print(accuracy_score(y_test, pred))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [13:35:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.87      0.90      0.89      5798
           1       0.92      0.89      0.90      2077
           2       0.93      0.90      0.91      2057
           3       0.89      0.91      0.90      3791
           4       1.00      0.99      0.99      1687
           5       0.97      0.97      0.97       655
           6       0.84      0.74      0.79       270
           7       0.94      0.92      0.93       878
           8       0.96      0.86      0.91      1125

    accuracy                           0.91     18338
   macro avg       0.92      0.90      0.91     18338
weighted avg       0.91      0.91      0.91     18338

0.9101864979823318


Для экономии времени были использованы предыдущие параметры полученные с помощью optune для модели, которая проявила себя наилучшим образом

In [118]:
Test = pd.read_csv("test_news.csv")
bow_test_c = bow.transform(Test['content'])
bow_test_c = scaler.transform(bow_test_c)
preds = xg_clf.predict(bow_test_c)
subm['topic'] = preds
subm.to_csv("xgup_lenta.csv", index=False)

К сожалению обучение на необработанных данных не дало какого либо прироста в метрике `accuracy`, которая была примерно равна на контрольных данных 0.824. Это, конечно, выше результатов, которые были получены в baseline модели, но как будет продемонстрированно далее, модель XGBoost все таки лучше работает на обработанных данных.

In [125]:
X = df[['processed_text']]
y = df.topic

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
bow_transformer = CountVectorizer(min_df=0.00001) # подбор гиперпараметров очень помогает
bow_transformer.fit(X_train['processed_text'])

bow_train = bow_transformer.transform(X_train['processed_text'])  # bow — bag of words (мешок слов)
bow_test = bow_transformer.transform(X_test['processed_text'])

print(bow_train.shape)

scaler = MaxAbsScaler()
bow_train = scaler.fit_transform(bow_train)
bow_test = scaler.transform(bow_test)

(55011, 148003)


In [123]:
# подбор параметров Optune
def xg_obj(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.5),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "scale_pos_weight": len(y_train) / (len(set(y_train)) * np.bincount(y_train)),
        "objective": "multi:softmax",  # Многоклассовая классификация
        "num_class": len(set(y_train)),  # Количество классов
        "random_state": 42,
        "use_label_encoder": False
    }
    
    # Создание модели
    clf = XGBClassifier(**param)

    # Обучение
    clf.fit(bow_train, y_train)

    # Предсказания
    pred = clf.predict(bow_test)

    # Оценка качества
    return accuracy_score(y_test, pred)

study = optuna.create_study(direction="maximize")
study.optimize(xg_obj, n_trials=20)

# Результаты
print("Лучшие параметры:", study.best_params)
print("Лучшая точность:", study.best_value)

In [127]:
spw = len(y_train) / (len(set(y_train)) * np.bincount(y_train))
params = {'n_estimators': 946, 'max_depth': 25, 'learning_rate': 0.02334486067381037,
          'subsample': 0.7571222430291574, 'colsample_bytree': 0.6866556282657983,
          'gamma': 0.09038470099513807, 'scale_pos_weight': spw}

xg_clf = XGBClassifier(**params, random_state=42, objective = "multi:softmax",
                       num_class = len(set(y_train)), use_label_encoder = False)
xg_clf.fit(bow_train, y_train)
pred = xg_clf.predict(bow_test)

print(classification_report(y_test, pred))
print(accuracy_score(y_test, pred))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:29:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.88      0.90      0.89      5798
           1       0.92      0.90      0.91      2077
           2       0.93      0.91      0.92      2057
           3       0.89      0.92      0.90      3791
           4       0.99      1.00      1.00      1687
           5       0.97      0.96      0.96       655
           6       0.87      0.78      0.82       270
           7       0.95      0.93      0.94       878
           8       0.95      0.88      0.91      1125

    accuracy                           0.91     18338
   macro avg       0.93      0.91      0.92     18338
weighted avg       0.91      0.91      0.91     18338

0.9132947976878613


In [129]:
Test = pd.read_csv("test_news_processed.csv")
bow_test_c = bow_transformer.transform(Test['processed_content'])
bow_test_c = scaler.transform(bow_test_c)
preds = xg_clf.predict(bow_test_c)
subm['topic'] = preds
subm.to_csv("xgcp_lenta.csv", index=False)

При обучении на обработанных данных метрика `accuracy` на контрольных данных была равна `0.83052`. Попробуем дообучить модель на всех данных. Возможно предсказания станут получше.

In [137]:
bow_transformer = CountVectorizer(min_df=0.00001) # подбор гиперпараметров очень помогает
bow_transformer.fit(X['processed_text'])

bow_X = bow_transformer.transform(X['processed_text'])  # bow — bag of words (мешок слов)

print(bow_X.shape)

scaler = MaxAbsScaler()
bow_X = scaler.fit_transform(bow_X)

(73349, 172390)


In [139]:
spw = len(y) / (len(set(y)) * np.bincount(y))
params = {'n_estimators': 946, 'max_depth': 25, 'learning_rate': 0.02334486067381037,
          'subsample': 0.7571222430291574, 'colsample_bytree': 0.6866556282657983,
          'gamma': 0.09038470099513807, 'scale_pos_weight': spw}

xg_clf = XGBClassifier(**params, random_state=42, objective = "multi:softmax",
                       num_class = len(set(y)), use_label_encoder = False)
xg_clf.fit(bow_X, y)
pred = xg_clf.predict(bow_X)

print(classification_report(y, pred))
print(accuracy_score(y, pred))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:55:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23191
           1       1.00      1.00      1.00      8309
           2       1.00      1.00      1.00      8228
           3       1.00      1.00      1.00     15163
           4       1.00      1.00      1.00      6749
           5       1.00      1.00      1.00      2620
           6       1.00      1.00      1.00      1079
           7       1.00      1.00      1.00      3511
           8       1.00      1.00      1.00      4499

    accuracy                           1.00     73349
   macro avg       1.00      1.00      1.00     73349
weighted avg       1.00      1.00      1.00     73349

1.0


In [141]:
Test = pd.read_csv("test_news_processed.csv")
bow_test_c = bow_transformer.transform(Test['processed_content'])
bow_test_c = scaler.transform(bow_test_c)
preds = xg_clf.predict(bow_test_c)
subm['topic'] = preds
subm.to_csv("xgcap_lenta.csv", index=False)

Результаты модели на контрольных данных чуть улучшились до значений метрики `accuracy = 0.83095`. Прирост не столь значим. Однако видно, что модель переобучена под данные. Возможно с этим можно что-то сделать. Попробуем применить L-2 регуляризацию для того, чтобы уменьшить переобучение.

In [152]:
spw = len(y) / (len(set(y)) * np.bincount(y))
params = {'n_estimators': 946, 'max_depth': 25, 'learning_rate': 0.02334486067381037,
          'subsample': 0.7571222430291574, 'colsample_bytree': 0.6866556282657983,
          'gamma': 0.09038470099513807, 'scale_pos_weight': spw}

xg_clf = XGBClassifier(**params, random_state=42, objective = "multi:softmax",
                       num_class = len(set(y)), use_label_encoder = False, reg_lambda = 10)
xg_clf.fit(bow_X, y)
pred = xg_clf.predict(bow_X)

print(classification_report(y, pred))
print(accuracy_score(y, pred))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:04:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23191
           1       1.00      1.00      1.00      8309
           2       1.00      1.00      1.00      8228
           3       1.00      1.00      1.00     15163
           4       1.00      1.00      1.00      6749
           5       1.00      1.00      1.00      2620
           6       1.00      1.00      1.00      1079
           7       1.00      1.00      1.00      3511
           8       1.00      1.00      1.00      4499

    accuracy                           1.00     73349
   macro avg       1.00      1.00      1.00     73349
weighted avg       1.00      1.00      1.00     73349

0.9993728612523688


In [154]:
Test = pd.read_csv("test_news_processed.csv")
bow_test_c = bow_transformer.transform(Test['processed_content'])
bow_test_c = scaler.transform(bow_test_c)
preds = xg_clf.predict(bow_test_c)
subm['topic'] = preds
subm.to_csv("xgcapl2_lenta.csv", index=False)

От переобучения к сожалению не удалось избавиться, а обучение модели занимает примерно минут 40, что довольно долго, если, например, для подбора параметров обучить где-то 25 моделей с помощью optune. Модель с L-2 регуляризацией дает метрики чуть хуже в плане метрики accuracy, которая равна примерно `0.82829`. Возможно я чуть позже попытаюсь еще раз подобрать параметры для улучшения результата, но это уже скорее будет идти как бонусный контент. Однако на текущий момент моделью, которая показывает себя наилучшим образом является переобученная модель XGBoost, которая показывает метрику accuracy, которая на 1.5% выше, чем у baseline модели. Это, конечно, не самый лучший результат, но добиться улучшения качества относительно baseline удалось.

## Выводы

- В ходе обучения моделей логистической регрессии было выявлено, что модель дает более лучшие результаты в предсказаниях на контрольных данных при обучении на сырых собранных данных, переработанных в мешок слов. Можно предположить, что данное явление вызвано тем, что модель логиситческой регресии довольно простая, и из-за этого она лучше работает с избыточной информацией. Также возможно данный результат вызван тем, что данные были обработаны излишне.
- В дальнейшем при обучении модели XGBoost ситуация была обратной. Модель показала себя лучше на обработанных данных. При этом эта модель модель показала себя лучше всего как на тренировочных данных, так и на контрольных данных после подбора гиперпараметров. Однако модель переобучается на тренировочных данных, что говорит о том, что в дальнейшем для получения лучших результатов вероятно стоит заново перебрать гиперпараметры, что весьма трудоемкий процесс часов на 10. Поэтому возможно это уже будет сделано после мягкого дедлайна уже в другом ноутбуке.
- Если говорить про то почему модель XGBoost показала себя лучше в плане метрик, то скорее всего этот результат вызван тем, что модель XGBoost является более мощной моделью и может улавливать нелинейные зависимости.
- Также вероятно на результаты могло повлиять возможно неправильное распределение новостей в кактегорию строительство, так как на сайте Lenta.ru отдельного раздела новостей с такой категорией нет, то пришлось немного извернуться и взять за эту категорию подраздел новостей связанных недвижимостью в экономическом разделе.
- Хоть в данный ноутбук и не были включены разделы с TF-IDF обработкой текста и SVM моделью, но это было сделано намеренно, чтобы не растягивать ноутбук. SVM модель не показала каких-либо значимых отличий в результатах по сравнению с логистической регрессией на мешке слов, а при применении TF-IDF, которая выделяет важность слов в тексте, модели хоти и начинали работать лучше на тренировочных данных, но на контрольных данных модели проявляли себя довольно плохо.